In [1]:
import numpy as np
from matplotlib import pyplot as plt
import random

In [2]:
X = np.load('X.npy')

# flatten the X matrix
X = X.reshape(X.shape[0], -1)

# # normalize the data
# min_value = np.min(X)
# max_value = np.max(X)
# X = ((X - min_value) / (max_value - min_value)) * 255
# X = np.round(X).astype(int)

Y = np.zeros((X.shape[0], 10))
Y[0:204, 9] = 1
Y[204:409, 0] = 1
Y[409:615, 7] = 1
Y[615:822, 6] = 1
Y[822:1028, 1] = 1
Y[1028:1236, 8] = 1
Y[1236:1443, 4] = 1
Y[1443:1649, 3] = 1
Y[1649:1855, 2] = 1
Y[1855:, 5] = 1

indices = np.argmax(Y, axis=1)
Y = np.expand_dims(indices, axis=1)

In [3]:
# Split data into train and test

data = np.concatenate((Y, X), axis=1)

np.random.shuffle(data)

Y = data[:, 0]
X = data[:, 1:]

X_train = X[0:2000]
Y_train = Y[0:2000]

X_test = X[2000:]
Y_test = Y[2000:]

Y_train = Y_train.reshape(Y_train.shape[0], 1)
Y_test = Y_test.reshape(Y_test.shape[0], 1)

In [4]:
def randInitializeWeights(L_in, L_out):
    """
    randomly initializes the weights of a layer with L_in incoming connections and L_out outgoing connections.
    """
    
    epi = (6**1/2) / (L_in + L_out)**1/2
    
    W = np.random.rand(L_out,L_in+1) *(2*epi) -epi
    
    return W

In [5]:
input_layer_size  = 4096
hidden_layer_size = 25
hidden_layer_size2 = 25
num_labels = 10

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, hidden_layer_size2)
initial_Theta3 = randInitializeWeights(hidden_layer_size2, num_labels)

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [7]:
def ReLU(z):
    return np.maximum(0, z)

In [8]:
#Computes the gradient of sigmoid function
def sigmoidGradient(z):
    """
    computes the gradient of the sigmoid function
    """
    sigGrad = sigmoid(z) * (1 - sigmoid(z))
    
    return sigGrad

In [9]:
def nnCostFunction(theta1, theta2, theta3, input_layer_size, hidden_layer_size, hidden_layer_size2, num_labels,X, y,Lambda):
    """
    nn_params contains the parameters unrolled into a vector
    
    compute the cost and gradient of the neural network
    """

    m = X.shape[0] # number of training examples = 2000
    J = 0
    y10 = np.zeros((m, num_labels)) # 2000x10
    for i in range(num_labels):
        y10[:,i][:,np.newaxis] = np.where(y==i,1,0)

    X = np.hstack((np.ones((m,1)),X)) # 2000x4097

    a1 = sigmoid(X @ theta1.T) # 2000x4097 * 4097x10 = 2000x10
    a1 = np.hstack((np.ones((m,1)), a1)) # hidden layer -> 2000x11
    a2 = sigmoid(a1 @ theta2.T) # output layer -> 2000x11 * 11x10 = 2000x10

    a2 = np.hstack((np.ones((m,1)), a2)) # hidden layer -> 2000x11

    a3 = sigmoid(a2 @ theta3.T) # output layer -> 2000x11 * 11x10 = 2000x10


    for j in range(num_labels):
        J = J + sum(-y10[:,j] * np.log(a3[:,j]) - (1-y10[:,j])*np.log(1-a3[:,j]))

    cost = 1/m * J
    reg_J = cost + Lambda/(2*m) * (np.sum(theta1[:,1:]**2) + np.sum(theta2[:,1:]**2) + np.sum(theta3[:,1:]**2))

    # Implement the backpropagation algorithm to compute the gradients

    grad1 = np.zeros((theta1.shape)) # 10x4097
    grad2 = np.zeros((theta2.shape)) # 10x11
    grad3 = np.zeros((theta3.shape)) # 10x11

    for i in range(m):
        xi = X[i,:]
        a1i = a1[i,:]
        a2i = a2[i,:]
        a3i = a3[i,:]
        d3 = a3i - y10[i,:]
        d2 = theta3.T @ d3.T * sigmoidGradient(np.hstack((1,xi @ theta1.T)))
        d1 = theta2.T @ d2[1:] * sigmoidGradient(np.hstack((1,xi @ theta1.T)))
        grad1 = grad1 + d1[1:][:,np.newaxis] @ xi[:,np.newaxis].T
        grad2 = grad2 + d2[1:][:,np.newaxis] @ a1i[:,np.newaxis].T
        grad3 = grad3 + d3.T[:,np.newaxis] @ a2i[:,np.newaxis].T

    grad1 = 1/m * grad1
    grad2 = 1/m * grad2
    grad3 = 1/m * grad3

    grad1_reg = grad1 + (Lambda/m) * np.hstack((np.zeros((theta1.shape[0],1)),theta1[:,1:]))
    grad2_reg = grad2 + (Lambda/m) * np.hstack((np.zeros((theta2.shape[0],1)),theta2[:,1:]))
    grad3_reg = grad3 + (Lambda/m) * np.hstack((np.zeros((theta3.shape[0],1)),theta3[:,1:]))

    return cost, grad1, grad2, grad3, reg_J, grad1_reg, grad2_reg, grad3_reg

In [10]:
def gradientDescentnn(X,y,initial_theta1, initial_theta2, initial_theta3, alpha,num_iters,Lambda,input_layer_size, input_layer_size2, hidden_layer_size, num_labels):
    """
    Take in numpy arra
    y X, y and theta and update theta by taking num_iters gradient steps
    with learning rate of alpha
    
    return theta and the list of the cost of theta during each iteration
    """
    m = X.shape[0]
    theta1 = initial_theta1
    theta2 = initial_theta2
    theta3 = initial_theta3
    J_history = []

    # v1 = np.zeros(theta1.shape)
    # v2 = np.zeros(theta2.shape)
    # v3 = np.zeros(theta3.shape)

    momentum = 0.8

    for i in range(num_iters):
        # theta1_head = theta1 - momentum * v1
        # theta2_head = theta2 - momentum * v2
        # theta3_head = theta3 - momentum * v3

        cost, grad1, grad2, rad3, reg_J, grad1_reg, grad2_reg, grad3_reg = nnCostFunction(theta1, theta2, theta3, input_layer_size, hidden_layer_size, hidden_layer_size2, num_labels,X,y,Lambda)
        # cost, grad1, grad2, rad3, reg_J, grad1_reg, grad2_reg, grad3_reg = nnCostFunction(theta1_head, theta2_head, theta3_head, input_layer_size, hidden_layer_size, hidden_layer_size2, num_labels,X,y,Lambda)
        # theta1 = theta1 - (alpha * grad1)
        # theta2 = theta2 - (alpha * grad2)

        theta1 = theta1 - (alpha * grad1_reg)
        theta2 = theta2 - (alpha * grad2_reg)
        theta3 = theta3 - (alpha * grad3_reg)

        # v1 = momentum * v1 + alpha * grad1_reg
        # v2 = momentum * v2 + alpha * grad2_reg
        # v3 = momentum * v3 + alpha * grad3_reg

        # theta1 = theta1 - v1
        # theta2 = theta2 - v2
        # theta3 = theta3 - v3

        if i % 10 == 0:
            print('Cost at iteration',i,':',cost)
            print('Regularized cost at iteration',i,':',reg_J)

            # alpha = alpha * 0.9
            # print('Learning rate at iteration',i,':',alpha)
            
        # J_history.append(cost)
        J_history.append(reg_J)

    return theta1, theta2, theta3, J_history

In [11]:
alpha = 0.1
num_iters = 1000
Lambda = 1
num_labels = 10

theta1, theta2, theta3, J_history = gradientDescentnn(X_train,Y_train,initial_Theta1, initial_Theta2, initial_Theta3,alpha,num_iters,Lambda,input_layer_size, hidden_layer_size, hidden_layer_size2, num_labels)

Cost at iteration 0 : 6.933957056286354
Regularized cost at iteration 0 : 6.934042552551208


In [ ]:
# Plot the cost function evolution during training.
#In order to say learning has finished, the cost function has to converge to a flat rate
plt.plot(J_history)  #
plt.xlabel("Iteration")
plt.ylabel("$J(\Theta)$")
plt.title("Cost function using Gradient Descent")

In [ ]:
def predict(theta1, theta2, theta3, X):
    """
    Predict the label of an input given a trained neural network
    """
    #number of training examples
    m= len(X)
        
    # add an extra column of 1´s corresponding to xo=1
    X = np.append(np.ones((m,1)),X,axis=1)
    
    #Compute the output of the hidden layer (with sigmoid activation functions)
    z1=np.dot(X, theta1.T)  #Inputs to the hidden layer neurons
    a1=sigmoid(z1)  #Outputs  of the hidden layer neurons
    
    #Add a column of ones
    a1 = np.append(np.ones((m,1)),a1, axis=1)
    
    #Compute the output of the output layer (with sigmoid activation functions)
    z2=np.dot(a1, theta2.T) #Inputs to the output layer neurons
    a2=sigmoid(z2)  #Outputs  of the output layer neurons

    #Add a column of ones
    a2 = np.append(np.ones((m,1)),a2, axis=1)

    #Compute the output of the output layer (with sigmoid activation functions)
    z3=np.dot(a2, theta3.T) #Inputs to the output layer neurons
    a3=sigmoid(z3)  #Outputs  of the output layer neurons

    #Predict the class
    return np.argmax(a3,axis=1)

In [ ]:
pred = predict(theta1, theta2, theta3, X_test)

print('Training Set Accuracy: {:.1f}%'.format(np.mean(pred == Y_test.ravel())*100))